In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用Dataproc和BigQuery的SparkML

<table align="left">

  <td>
<a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/workbench/spark/spark_ml.ipynb" target='_blank'>
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      查看GitHub上的代码
    </a>
  </td>
  <td>
        <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/workbench/spark/spark_ml.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
        </a>
  </td>
  <td>
<a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/workbench/spark/spark_ml.ipynb" target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>                                                                                               
</table>

## 概览

本笔记本教程运行Apache SparkML作业与Dataproc和BigQuery，以示例一个常见的机器学习流水线用例：数据摄入和清洗，特征工程，建模和模型评估。

了解有关[Vertex AI Workbench](https://cloud.google.com/vertex-ai/docs/workbench/introduction)和[Dataproc](https://cloud.google.com/vertex-ai/docs/pipelines/dataproc-component)的更多信息。

### 目标

本教程运行一个Apache SparkML作业，从BigQuery数据集中获取数据，执行探索性数据分析，清洗数据，执行特征工程，训练模型，评估模型，输出结果，并将模型保存到Cloud Storage存储桶中。

本教程使用以下Google Cloud ML服务：

- `Dataproc`
- `BigQuery`
- `Vertex AI Training`

执行的步骤包括：

- 设置一个Google Cloud项目和Dataproc集群。
- 创建一个Cloud Storage存储桶和一个BigQuery数据集。
- 配置spark-bigquery-connector。
- 将BigQuery数据导入Spark DataFrame中。
- 进行探索性数据分析（EDA）。
- 使用样本可视化数据。
- 清洗数据。
- 选择特征。
- 训练模型。
- 输出结果。
- 将模型保存到Cloud Storage存储桶中。
- 删除为本教程创建的资源。

数据集

[NYC TLC（纽约出租车和豪华轿车委员会）行程](https://console.cloud.google.com/marketplace/product/city-of-new-york/nyc-tlc-trips)（纽约出租车和豪华轿车行程数据）和[NYC Citi Bike行程](https://console.cloud.google.com/marketplace/product/city-of-new-york/nyc-citi-bike)（纽约市公共自行车共享系统数据）数据集可在[BigQuery公共数据集](https://cloud.google.com/bigquery/public-data)中获得。 BigQuery每个月提供免费查询高达1TB的数据。

### 费用

此教程使用 Google Cloud 的计费组件：

* [Vertex AI](https://cloud.google.com/vertex-ai/pricing)
* [Cloud Storage](https://cloud.google.com/storage/pricing)
* [Dataproc](https://cloud.google.com/dataproc/pricing)
* [BigQuery](https://cloud.google.com/bigquery/pricing)

您可以使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用量生成费用估算。

### 安装

安装以下包:

In [ ]:
import os

if os.getenv("IS_TESTING"):
    """
    Since the testing suite doesn't support testing on Dataproc clusters,
    the testing environment is setup to replicate Dataproc via the following steps:
    """
    JAVA_VER = "8u332-b09"
    JAVA_FOLDER = "/tmp/java"
    FILE_NAME = f"openlogic-openjdk-{JAVA_VER}-linux-x64"
    TAR_FILE = f"{JAVA_FOLDER}/{FILE_NAME}.tar.gz"
    DOWNLOAD_LINK = f"https://builds.openlogic.com/downloadJDK/openlogic-openjdk/{JAVA_VER}/openlogic-openjdk-{JAVA_VER}-linux-x64.tar.gz"
    PYSPARK_VER = "3.1.3"

    ! rm -rf $JAVA_FOLDER
    ! mkdir $JAVA_FOLDER
    # Download Open JDK 8. Spark requires Java to execute.
    ! wget -P $JAVA_FOLDER $DOWNLOAD_LINK
    os.environ["JAVA_HOME"] = f"{JAVA_FOLDER}/{FILE_NAME}"
    ! tar -zxf $TAR_FILE -C $JAVA_FOLDER
    ! echo $JAVA_HOME

    # Install PySpark and other packages.
    ! pip install pyspark==$PYSPARK_VER geopandas pyarrow rtree seaborn numpy==1.19.5 -q

### 创建一个 Dataproc 集群

在这个笔记本教程中执行的 Spark 作业计算密集。由于在标准笔记本环境中完成作业可能需要较长时间，因此这个笔记本教程在一个使用了 Dataproc Component Gateway 和 Jupyter 组件的 Dataproc 集群上运行。

**已有 Dataproc 和 Jupyter 集群？**：如果你已经有一个正在运行的 Dataproc 集群，并且该集群上已安装了 [Component Gateway 和 Jupyter 组件](https://cloud.google.com/dataproc/docs/concepts/components/jupyter#gcloud-command)，你可以在这个教程中使用它。如果您计划使用它，请跳过这一步，转到 `切换内核`。

为您的新集群设置一个名称和 [计算区域](https://cloud.google.com/compute/docs/regions-zones#available)。您的 `CLUSTER_NAME` 必须在您的 Google Cloud 项目中是**唯一的**。它必须以小写字母开头，后跟最多 51 个小写字母、数字和连字符，并且不能以连字符结尾。

In [ ]:
# The testing environment does not use a Dataproc cluster so cluster creation is skipped during testing.
if not os.getenv("IS_TESTING"):
    CLUSTER_NAME = "[your-cluster]"  # @param {type: "string"}
    CLUSTER_REGION = "[your-region]"  # @param {type: "string"}

    if CLUSTER_REGION == "[your-region]":
        CLUSTER_REGION = "us-central1"

    print(f"CLUSTER_NAME: {CLUSTER_NAME}")
    print(f"CLUSTER_REGION: {CLUSTER_REGION}")

In [ ]:
if not os.getenv("IS_TESTING"):
    !gcloud dataproc clusters create $CLUSTER_NAME \
        --region=$CLUSTER_REGION \
        --enable-component-gateway \
        --image-version=2.0 \
        --optional-components=JUPYTER

#### 切换内核

您的笔记本内核列在笔记本页面顶部。您的笔记本应该在运行在您的Dataproc集群上的Python 3内核上运行。

从顶部菜单中选择**内核 > 更改内核**，然后选择`Python 3 on CLUSTER_NAME: Dataproc 集群 in REGION (远程)`。

### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### 地区

您也可以更改 Vertex AI 使用的 `REGION` 变量。了解有关 [Vertex AI 地区](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "us-central1"

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

# If you do not specify a region, it is set to "us-central1".
if not REGION or REGION == "" or REGION == "[your-region]":
    REGION = "us-central1"

UUID（Universally Unique Identifier）

为了避免名称冲突，您可以为当前笔记本会话创建一个UUID，然后将该UUID附加到在本教程中创建的资源（如云存储桶和BigQuery数据集）的名称后面。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 验证您的谷歌云账户

根据您的Jupyter环境，您可能需要手动进行身份验证。请按照下面的相关说明进行操作。

**1. Vertex AI Workbench**
* 无需操作，因为您已经通过验证。

**2. 本地JupyterLab实例，请取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

请参考如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples上为您的服务帐号授予云存储权限。

创建一个云存储桶

创建一个存储桶来存储中间产物，例如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有当您的桶尚不存在时才能运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

## 教程

### 导入所需的库

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from geopandas import gpd
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import GBTRegressor
# A Spark Session is how you interact with Spark SQL to create Dataframes
from pyspark.sql import SparkSession
# PySpark functions
from pyspark.sql.functions import (col, floor, pandas_udf, to_timestamp, udf,
                                   unix_timestamp)
# These allow us to create a schema for our data
from pyspark.sql.types import BooleanType, DoubleType
from shapely.geometry import Point

请注意：在导入库之后，如果看到`ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/miniconda3/share/proj failed`，您可以忽略它。这是由于`geopandas`依赖存在错误。

初始化SparkSession

要在Apache Spark中使用BigQuery，您必须在初始化SparkSession时包含[spark-bigquery-connector](https://github.com/GoogleCloudDataproc/spark-bigquery-connector)。

In [ ]:
VER = "0.26.0"
FILE_NAME = f"spark-bigquery-with-dependencies_2.12-{VER}.jar"

if os.getenv("IS_TESTING"):
    # The local testing environment is not configured to read GCS files using Spark. This is a workaround for testing.
    connector = f"https://github.com/GoogleCloudDataproc/spark-bigquery-connector/releases/download/{VER}/{FILE_NAME}"
else:
    connector = f"gs://spark-lib/bigquery/{FILE_NAME}"


# Initialize the SparkSession.
spark = (
    SparkSession.builder.appName("spark-ml-taxi-citibike")
    .config("spark.jars", connector)
    .getOrCreate()
)

### 从BigQuery获取数据

本教程使用了2017年纽约市出租车数据集和2013年至2018年的纽约市自行车数据集。由于每个数据集的时间范围不同，将两个数据集匹配到2017年的时间段。

In [ ]:
# Load NYC_taxi in Github Activity Public Dataset from BigQuery.
taxi_df = (
    spark.read.format("bigquery")
    .option("table", "bigquery-public-data.new_york_taxi_trips.tlc_green_trips_2017")
    .load()
)


# Load NYC_Citibike in Github Activity Public dataset from BQ.
bike_df = (
    spark.read.format("bigquery")
    .option("table", "bigquery-public-data.new_york_citibike.citibike_trips")
    .load()
)

使用`to_timestamp()`和`unix_timestamp()`函数将`starttime`和`stoptime`转换为Unix时间戳类型。

Unix时间戳是一个整数，表示从`1970年1月1日00:00:00 UTC`开始的秒数，易于解析并在不同系统之间使用。

In [ ]:
# Represents 2017-01-01 00:00:00
START_2017 = 1483228800
# Represents 2017-12-31 23:59:59
END_2017 = 1514764799

# Convert the type of starttime from a string to a Unix timestamp.
bike_df = bike_df.withColumn(
    "starttime", unix_timestamp(to_timestamp(col("starttime")))
)

# Convert the type of stoptime from a string to a Unix timestamp.
bike_df = bike_df.withColumn("stoptime", unix_timestamp(to_timestamp(col("stoptime"))))

# Filter data so that `bike_df` only contains 2017 trips.
bike_df = bike_df.where(
    (col("starttime") >= START_2017)
    & (col("starttime") <= END_2017)
    & (col("stoptime") >= START_2017)
    & (col("stoptime") <= END_2017)
)

"""
Since notebook runtime scales linearly with the size of data, to limit processing time and compute resource consumption, 
this tutorial uses 20% of the Citi Bike dataset. You can try different values or remove the next line to use all of the data. 
"""
bike_df = bike_df.sample(0.2)

if os.getenv("IS_TESTING"):
    taxi_df = taxi_df.sample(0.0001)
    bike_df = bike_df.sample(0.0001)

### 进行探索性数据分析（EDA）

作为第一步，使用探索性数据分析（EDA）通过图表和其他可视化方法来发现数据中的模式、关系和异常。

首先检查出租车数据集的数据类型。

In [ ]:
taxi_df.printSchema()

筛选掉不必要的列，并检查字段的空值计数。请注意，`pickup_location_id`和`dropoff_location_id`已从BigQuery数据集转换为字符串格式。为了有效使用它们，请重新格式化这些值。

In [ ]:
taxi_df = taxi_df.select(
    "pickup_datetime",
    "dropoff_datetime",
    "trip_distance",
    "fare_amount",
    col("pickup_location_id").cast("int").alias("start_zone_id"),
    col("dropoff_location_id").cast("int").alias("end_zone_id"),
)

taxi_df.describe().show()
taxi_df.show()

从这个摘要中，你可以了解到很多信息。

- 2017年绿色出租车的行程历史记录超过1100万条，其中包括一些异常值，比如负值。
- 直到2016年，精确的纬度和经度被用来表示出租车的上车和下车位置。这些数据引起了[隐私关注](https://agkn.wordpress.com/2014/09/15/riding-with-the-stars-passenger-privacy-in-the-nyc-taxicab-dataset/)。数据集中的`pickup_location_id`和`dropoff_location_id`对应于[纽约出租车区域](https://data.cityofnewyork.us/Transportation/NYC-Taxi-Zones/d3c5-ddgc)，大致以纽约市规划部门的社区统计地区（NTAs）为基础，并用来表示大致的社区位置。

根据上述数据，过滤掉不现实的值。

- 删除空值。
- 行程距离必须在0到20英里之间，因为曼哈顿大约长13.4英里。
- 车费金额，不包括税和小费，必须在0到500美元之间。

In [ ]:
taxi_df = taxi_df.where(
    (col("trip_distance") > 0)
    & (col("trip_distance") < 20)
    & (col("fare_amount") > 0)
    & (col("fare_amount") < 500)
).dropna()

使用`unix_timestamp()`函数将`pickup_datetime`和`dropoff_datetime`转换为Unix Timestamp类型。

将`pickup_datetime`和`dropoff_datetime`转换为`start_time`和`end_time`后，您可以操作这些时间戳生成更多有趣的列数据。

In [ ]:
# Convert the type of datetime from a string to a Unix timestamp.
taxi_df = taxi_df.withColumn("start_time", unix_timestamp(col("pickup_datetime")))
taxi_df = taxi_df.withColumn("end_time", unix_timestamp(col("dropoff_datetime")))

# Calculate if this is a weekday
taxi_df = taxi_df.withColumn(
    "is_weekday",
    ((floor(col("start_time") / 86400) + 4) % 7 > 0)
    & ((floor(col("start_time") / 86400) + 4) % 7 < 6),
)

# Convert start_time to start_time_in_minute.
taxi_df = taxi_df.withColumn(
    "start_time_in_hour", floor((col("start_time") % 86400) / 60 / 60)
)

# Calculate trip_duration.
taxi_df = taxi_df.withColumn("trip_duration", col("end_time") - col("start_time"))

### 进行特征工程

出租车数据集包含纽约市所有行政区的行程，而 Citi Bike 数据集包含曼哈顿和布鲁克林以及布朗克斯部分地区之间的行程。为了匹配位置，过滤掉除了曼哈顿以外的所有行政区。

为了将位置 ID 转换为行政区，可以从 NYC OpenData 下载纽约市出租车区域的 GeoJSON 格式数据。

In [ ]:
# Load the GeoJSON format of NYC Taxi zones. If a URLError is thrown, please re-run this cell.
gdf_zone = gpd.read_file(
    "https://data.cityofnewyork.us/api/geospatial/d3c5-ddgc?method=export&format=GeoJSON"
)

In [ ]:
# Convert the type of "location_id" column.
gdf_zone["location_id"] = gdf_zone["location_id"].astype("long")

# Convert GeoPandas format to a dictionary.
hm = gdf_zone.to_dict("index")

# Create a set that contains location_id if the location is in Manhattan.
manhattan_set = {hm[i]["location_id"] for i in hm if hm[i]["borough"] == "Manhattan"}


@udf(returnType=BooleanType())
def is_in_manhattan(location_id):
    """
    The preprocessing function takes location_id and returns whether the given location_id is in the manhattan_set.
    Args:
        location_id: A number from 0 to 263 that represents NYC Taxi Zone.
    Returns:
        A Boolean value of whether the given location_id is in the manhattan_set.
    """
    return location_id in manhattan_set

In [ ]:
# Trips occur within Manhattan and between different taxi zones.
taxi_df = taxi_df.where(
    (col("start_zone_id") != col("end_zone_id"))
    & (is_in_manhattan(col("start_zone_id")))
    & (is_in_manhattan(col("end_zone_id")))
)

In [ ]:
taxi_df.show()
taxi_df.describe().show()

在可视化这个修改后的出租车数据集之前，为Citi Bike数据集进行类似的特征工程处理。

In [ ]:
bike_df.printSchema()
bike_df.describe().show()

这个数据集显示了一些有趣的信息：

* 2017年Citi Bike的行程记录超过了280万条。
* 当前数据集存在一些异常值。
* `starttime`和`stoptime`是以字符串格式存在的。要有效使用它们，需要重新格式化。
* Citi Bike数据集中不包含NYC出租车区域的`location_id`，只有坐标数据。想要将坐标转换成NYC出租车区域的话，可以使用Pandas UDF，一个矢量化UDF特性（详见[Introducing Pandas UDF for PySpark](https://www.databricks.com/blog/2017/10/30/introducing-vectorized-udfs-for-pyspark.html)）。

In [ ]:
@pandas_udf("long")
def preprocess_zone_id(lat: pd.Series, lon: pd.Series) -> pd.Series:
    """
    The preprocessing function takes latitudes and longitudes and returns the corresponding Taxi Zone id.
    Args:
        lat: The latitude of the position.
        lon: The longitude of the position.
    Returns:
        The Taxi Zone id.
    """
    point_var = [Point(xy) for xy in zip(lon, lat)]
    gdf_points = gpd.GeoDataFrame(
        pd.DataFrame({"lat": lat, "lon": lon}), crs="epsg:4326", geometry=point_var
    )
    gdf_joined = gpd.sjoin(gdf_points, gdf_zone, how="left")
    return gdf_joined["location_id"]

In [ ]:
# Rename "tripduration" to "trip_duration".
bike_df = bike_df.withColumnRenamed("tripduration", "trip_duration")

# Check whether the starttime is a weekday or a weekend.
bike_df = bike_df.withColumn(
    "is_weekday",
    ((floor(col("starttime") / 86400) + 4) % 7 > 0)
    & ((floor(col("starttime") / 86400) + 4) % 7 < 6),
)

# Convert starttime to start_time_in_minute
bike_df = bike_df.withColumn(
    "start_time_in_hour", floor((col("starttime") % 86400) / 60 / 60)
)

# Add "start_zone_id" and "end_zone_id" by applying the UDF "preprocess_zone_id".
bike_df = bike_df.withColumn(
    "start_zone_id",
    preprocess_zone_id(
        bike_df["start_station_latitude"], bike_df["start_station_longitude"]
    ),
)
bike_df = bike_df.withColumn(
    "end_zone_id",
    preprocess_zone_id(
        bike_df["end_station_latitude"], bike_df["end_station_longitude"]
    ),
)

# Filter `bike_df` using the following conditions:
# 1) Trips must done between different taxi zones.
# 2) The start time must be earlier than the end time.
# 3) Trips must start and end within Manhattan.
bike_df = bike_df.where(
    (col("start_zone_id") != col("end_zone_id"))
    & (col("starttime") < col("stoptime"))
    & (is_in_manhattan(col("start_zone_id")))
    & (is_in_manhattan(col("end_zone_id")))
)

bike_df = bike_df.select(
    "trip_duration",
    "starttime",
    "stoptime",
    "start_station_latitude",
    "start_station_longitude",
    "end_station_latitude",
    "end_station_longitude",
    "usertype",
    "start_zone_id",
    "end_zone_id",
    "is_weekday",
    "start_time_in_hour",
)

假设 Citi Bike 用户穿行在曼哈顿的街道上，街道是垂直的，通过应用曼哈顿距离可以计算行程的距离。

曼哈顿距离计算示例：假设你在**第15街和第9大道，即Google纽约位于的地方**，你要去**第33街和第5大道，即帝国大厦所在地**。你从第15街向东移动，直到到达第5大道，然后继续向北行驶，直到到达第33街。在这种情况下，**第16街和第5大道**或**第33街和第9大道**可以作为铰接点。如果我们将起点设置为**S**，终点设置为**E**，铰接点设置为**H**，则曼哈顿距离的公式为 `距离(S, H) + 距离(H, E)`。

然而，需要进行额外的计算才能获得“真实”的曼哈顿距离。首先，与笛卡尔坐标系不同，经度上的实际距离随纬度而变化。例如，在赤道，1度经度大约代表69英里，而在北纬或南纬45度时，大约代表49英里。要利用纬度和经度计算现实世界的确切距离，请使用[Haversine距离计算](https://en.wikipedia.org/wiki/Haversine_formula)。

此外，曼哈顿街道与真正的北方相差大约29度，因此点应该逆时针旋转29度。

因此，行程距离的公式应为`曼哈顿距离 = Haversine距离(S'，H') + Haversine距离(H'， E')`，其中`S'`，`H'`，`E'` 是旋转后的点。

In [ ]:
@udf(returnType=DoubleType())
def manhattan_dist(lat_start, lon_start, lat_end, lon_end):
    """
    The preprocessing function takes the latitude and longitude of start position and end position,
    and returns the Manhattan distance between them.
    Args:
        lat_start: The latitude of the start position.
        lon_start: The longitude of the start position.
        lat_end: The latitude of the end position.
        lon_end: The longitude of the end position.
    Returns:
        The Manhattan distance between start and end position.
    """
    # Approximate radius of Earth in mile.
    EARTH_RADIUS = 3958.76

    # Approximate inclined degree of the streets in Manhattan.
    THETA = np.radians(-28.904)

    def haversine(pos_start, pos_end):
        """
        The helper function takes start and end position and returns the haversine distance.
        Args:
            pos_start: a latitude and a longitude of start position in np.array form.
            pos_end: a latitude and a longitude of end position in np.array form.
        Returns:
            The Haversine, the spherical distance between `pos_start` and `pos_end` on a sphere, in this case, the earth.
        """
        rad_dist_lat = np.radians(pos_end[0] - pos_start[0])
        rad_dist_lon = np.radians(pos_end[1] - pos_start[1])
        rad_start_lat = np.radians(pos_start[0])
        rad_end_lat = np.radians(pos_end[0])
        dist = 2 * np.arcsin(
            np.sqrt(
                np.sin(rad_dist_lat / 2) ** 2
                + np.cos(rad_start_lat)
                * np.cos(rad_end_lat)
                * np.sin(rad_dist_lon / 2) ** 2
            )
        )
        return EARTH_RADIUS * dist

    def rotate(pos, theta):
        """
        The helper function takes position and the degree theta, and returns rotated position.
        Args:
            pos: a latitude and a longitude of position in np.array form.
            theta: the degree to rotate.
        Returns:
            Rotated position.
        """
        rotate = np.array(
            [[np.cos(theta), np.sin(theta)], [-np.sin(theta), np.cos(theta)]]
        )
        return np.matmul(rotate, pos)

    if not (lat_start and lon_start and lat_end and lon_end):
        return -1

    # Convert positions to np.array format.
    start, end = np.array([lat_start, lon_start]), np.array([lat_end, lon_end])

    # Rotate each positions by 29' using a helper function.
    rotated_start, rotated_end = rotate(start, THETA), rotate(end, THETA)

    # Get rotated hinge point using rotated start and end point.
    rotated_hinge = np.array([rotated_start[0], rotated_end[1]])

    # Re-rotate the hinge point.
    hinge = rotate(rotated_hinge, -THETA)

    # Return the sum of the Haversine distance between start and hinge, hinge and end.
    return float(haversine(start, hinge) + haversine(hinge, end))

In [ ]:
# Calculate the Manhattan distance between "start_station" and "end_station".
bike_df = bike_df.withColumn(
    "trip_distance",
    manhattan_dist(
        col("start_station_latitude"),
        col("start_station_longitude"),
        col("end_station_latitude"),
        col("end_station_longitude"),
    ),
)

In [ ]:
# Filter out the bike_df with the same condition as the taxi_df
bike_df = bike_df.where(
    (col("trip_distance") > 0)
    & (col("trip_distance") < 20)
    & (col("trip_duration") > 0)
    & (col("start_station_latitude") > 40)
    & (col("start_station_latitude") < 41)
    & (col("end_station_latitude") > 40)
    & (col("end_station_latitude") < 41)
    & (col("start_station_longitude") > -75)
    & (col("start_station_longitude") < -73)
    & (col("end_station_longitude") > -75)
    & (col("end_station_longitude") < -73)
)

In [ ]:
bike_df.printSchema()

检查数字列的分布。在PySpark中，可视化是昂贵的，因为数据量很大。

例如，纽约出租车数据集有超过1000万行。因此，总数据的约3%（约330k行）被提取为样本。

In [ ]:
if not os.getenv("IS_TESTING"):
    taxi_pd = taxi_df.sample(0.03).toPandas()
else:
    taxi_pd = taxi_df.toPandas()

# Convert "object" type columns to the float type.
taxi_pd["trip_distance"] = taxi_pd["trip_distance"].astype(float)
taxi_pd["fare_amount"] = taxi_pd["fare_amount"].astype(float)
taxi_pd.info()

if not os.getenv("IS_TESTING"):
    bike_pd = bike_df.sample(0.03).toPandas()
else:
    bike_pd = bike_df.toPandas()
bike_pd.info()

去除极端数据点，将数据分成箱和直方图。

In [ ]:
TAXI_COLUMNS_TO_SHOW = {"trip_distance", "trip_duration", "fare_amount"}

taxi_pd_filtered = taxi_pd.query(
    "trip_distance > 0 and trip_distance < 20 \
    and trip_duration > 0 and trip_duration < 10000"
)

sns.relplot(
    data=taxi_pd_filtered,
    x="trip_distance",
    y="trip_duration",
    kind="scatter",
)

for column in taxi_pd_filtered.columns:
    if column in TAXI_COLUMNS_TO_SHOW:
        _, ax = plt.subplots(1, 2, figsize=(10, 4))
        taxi_pd_filtered[column].plot(kind="box", ax=ax[0])
        taxi_pd_filtered[column].plot(kind="hist", ax=ax[1])
        plt.title(column)
        plt.figure()
plt.show()

大多数列向右倾斜，'trip_distance'随着'trip_duration'的增加而增加。大多数行程在一个小时内完成（3600秒）。

对Citi Bike数据集进行类似的分析。

In [ ]:
BIKE_COLUMNS_TO_SHOW = {"trip_distance", "trip_duration"}

bike_pd_filtered = bike_pd.query(
    "trip_distance > 0 and trip_distance < 20 \
    and trip_duration > 0 and trip_duration < 10000"
)

sns.relplot(
    data=bike_pd_filtered,
    x="trip_distance",
    y="trip_duration",
    hue="usertype",
    kind="scatter",
)

Citi Bike的“trip_distance”和“trip_duration”之间的关系与出租车数据集中的相同图表不同。

许多行程持续时间比预期的要长。这种行为的显而易见原因是：1）与出租车乘客不同，共享单车骑手不太可能有特定的行程目的地，2）偶尔使用Citi Bike的用户不会将他们的自行车停放在站点附近。

为了去除离群值，我们假设平均自行车速度至少为每小时2英里。我们将筛选出“trip_duration”乘以速度小于“trip_distance”的行程，并移除“Customer”用户类型。

In [ ]:
bike_pd_filtered = bike_pd.query(
    "trip_distance > 0 and trip_distance < 8 \
    and (trip_duration * 0.0005) <= trip_distance \
    and trip_duration > 0 and trip_duration < 10000 \
    and usertype=='Subscriber'"
)

sns.relplot(
    data=bike_pd_filtered,
    x="trip_distance",
    y="trip_duration",
    hue="usertype",
    kind="scatter",
)

for column in bike_pd_filtered.columns:
    if column in BIKE_COLUMNS_TO_SHOW:
        _, ax = plt.subplots(1, 2, figsize=(10, 4))
        bike_pd_filtered[column].plot(kind="box", ax=ax[0])
        bike_pd_filtered[column].plot(kind="hist", ax=ax[1])
        plt.title(column)
        plt.figure()
plt.show()

基于可视化的数据，应用过滤器来去除不现实的值。

In [ ]:
taxi_df = taxi_df.where(
    (col("trip_duration") > 0)
    & (col("trip_duration") < 3600)
    & ((col("trip_duration") * 0.00055) <= col("trip_distance"))
)

bike_df = bike_df.where(
    (col("trip_duration") > 0)
    & (col("trip_duration") < 3600)
    & (col("usertype") == "Subscriber")
    & ((col("trip_duration") * 0.00055) <= col("trip_distance"))
)

特征选择

我们的数据集中并非所有特征都是有用的。
由于本教程的目的是比较两个数据集，我们需要在训练中使用相同的特征。对于训练出租车数据集而言，有一些有用的特征，比如`fare_amount`，但由于Citi Bike数据集中没有这个特征，本教程中将不会使用它。

在选择了以下列作为特征后，它们必须使用`VectorAssembler()`进行组装，这是一种特征转换器，可以将多个列合并成一个向量列。

In [ ]:
feature_cols = [
    "is_weekday",
    "start_time_in_hour",
    "start_zone_id",
    "end_zone_id",
    "trip_distance",
]

assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

# Transform each column into the vector form.
taxi_transformed_data = assembler.transform(taxi_df)
bike_transformed_data = assembler.transform(bike_df)

# Split randomly with training and test data.
(taxi_training_data, taxi_test_data) = taxi_transformed_data.randomSplit([0.95, 0.05])
(bike_training_data, bike_test_data) = bike_transformed_data.randomSplit([0.95, 0.05])

### 训练模型

PySpark提供了几种回归模型，包括`LinearRegression`、`GeneralizedLinearRegression`、`DecisionTreeRegressor`、`RandomForestRegressor`和`GBTRegressor`。
`GBTRegressor`，也称为`Gradient Boosted Trees`，是一种流行的回归方法，使用决策树的集成。该算法通过迭代训练决策树以最小化损失函数。
尽管这个模型计算成本很高，但测试表明`GBTRegressor`提供了最佳结果。

In [ ]:
# Define GBTRegressor
gbt = GBTRegressor(
    featuresCol="features",
    labelCol="trip_duration",
    predictionCol="pred_trip_duration",
)

# Define evaluator for r2 score.
evaluator_r2 = RegressionEvaluator(
    labelCol=gbt.getLabelCol(), predictionCol=gbt.getPredictionCol(), metricName="r2"
)

# Define evaluator for RMSE error.
evaluator_rmse = RegressionEvaluator(
    labelCol=gbt.getLabelCol(), predictionCol=gbt.getPredictionCol(), metricName="rmse"
)

In [ ]:
# Train a model and get predictions for the Taxi dataset. It takes several minutes.
taxi_gbt_model = gbt.fit(taxi_training_data)
taxi_gbt_predictions = taxi_gbt_model.transform(taxi_test_data)

In [ ]:
# Train a model and get predictions for the Citi Bike dataset. It takes several minutes.
bike_gbt_model = gbt.fit(bike_training_data)
bike_gbt_predictions = bike_gbt_model.transform(bike_test_data)

In [ ]:
# Evaluate the r2 score for the Taxi dataset
taxi_gbt_accuracy_r2 = evaluator_r2.evaluate(taxi_gbt_predictions)
print(f"Taxi Test GBT R2 Accuracy = {taxi_gbt_accuracy_r2}")

# Evaluate the RMSE for the Taxi dataset
taxi_gbt_accuracy_rmse = evaluator_rmse.evaluate(taxi_gbt_predictions)
print(f"Taxi Test GBT RMSE Accuracy = {taxi_gbt_accuracy_rmse}")

In [ ]:
# Evaluate the r2 score for the Citi Bike dataset
bike_gbt_accuracy_r2 = evaluator_r2.evaluate(bike_gbt_predictions)
print(f"Bike Test GBT R2 Accuracy = {bike_gbt_accuracy_r2}")

# Evaluate the RMSE for the Citi Bike dataset
bike_gbt_accuracy_rmse = evaluator_rmse.evaluate(bike_gbt_predictions)
print(f"Bike Test GBT RMSE Accuracy = {bike_gbt_accuracy_rmse}")

查看结果

对于出租车数据集训练的模型显示大约77%的r2分数和230的均方根误差（RMSE），而Citi Bike数据集的模型显示大约71%的r2分数和250的RMSE。这些相对较低分数有一些原因。

* 没有使用CrossValidator

  * 在真实的机器学习项目中，交叉验证是一种重要工具，使我们能更好地利用数据。然而，它会多次重新运行训练算法，因此需要大量的时间和计算资源来完成。有关更多信息，请参见[交叉验证（统计学）](https://en.wikipedia.org/wiki/Cross-validation_(statistics))。

* 没有使用出租车数据集中的每个特征。

  * 出租车数据集有一个`fare_amount`列，它与标签`trip_duration`之间有很强的相关性。如果包含该列，出租车数据集的模型显示90%以上的r2分数。
  * 出租车数据集特征有限的原因是这个项目的目的是比较出租车和Citi Bike。

* 没有使用两个数据集的精确坐标。

  * 由于隐私问题，出租车数据集不再提供接送地点的精确位置，只提供纽约出租车区域的位置。
  * 出租车区域是一个表示纽约出租车定义的地理区域的分类号码。

将模型保存到云存储路径###

In [ ]:
BUCKET_FOLDER = "/tmp/bucket"

# In the testing environment, saving to Cloud Storage bucket routes to the local file system.
if os.getenv("IS_TESTING"):
    ! rm -rf $BUCKET_FOLDER
    ! mkdir $BUCKET_FOLDER
    bike_gbt_model.write().overwrite().save(f"{BUCKET_FOLDER}")
    ! gsutil cp -r $BUCKET_FOLDER $BUCKET_URI
else:
    bike_gbt_model.write().overwrite().save(f"{BUCKET_URI}/")

清理

请参考[清理](https://cloud.google.com/vertex-ai/docs/workbench/managed/create-managed-notebooks-instance-console-quickstart#clean-up)来删除此教程中创建的项目或托管笔记本。

### 删除Google Cloud Storage存储桶

In [ ]:
import os

DELETE_BUCKET = True

if DELETE_BUCKET or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI

### 删除Dataproc集群

除非将内核切换到本地，否则无法删除当前正在使用的集群。要删除它，您需要将内核切换到本地的`Python 3`或`PySpark`，在下面的单元格中手动设置`CLUSTER_NAME`和`CLUSTER_REGION`，然后执行`gcloud`命令。

请参阅[删除集群](https://cloud.google.com/dataproc/docs/guides/manage-cluster#console)以删除本教程中创建的Dataproc集群。

In [ ]:
CLUSTER_NAME = "[your-cluster-name]"
CLUSTER_REGION = "[your-cluster-region]"

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    ! gcloud dataproc clusters delete $CLUSTER_NAME --region=$CLUSTER_REGION -q